In [1]:
# Cell 1: Imports + build simple sequence dataset

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# دنباله هدف: ABCABCABC...
text = "ABC" * 100  # طول دنباله را می‌توانی بیشتر هم بکنی

chars = sorted(list(set(text)))   # ['A', 'B', 'C']
vocab_size = len(chars)

print("کاراکترها:", chars)
print("طول متن:", len(text))

# مپ کردن کاراکتر ↔ اندیس
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for ch, i in char2idx.items()}

# تبدیل کل متن به اندیس
encoded = np.array([char2idx[ch] for ch in text])

# ساخت جفت‌های (ورودی، خروجی)
seq_len = 5   # مثلا 5 کاراکتر ورودی → کاراکتر بعدی خروجی
X = []
y = []

for i in range(len(encoded) - seq_len):
    X.append(encoded[i:i+seq_len])
    y.append(encoded[i+seq_len])

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)


کاراکترها: ['A', 'B', 'C']
طول متن: 300
X shape: (295, 5)
y shape: (295,)


In [2]:
# Cell 2: Prepare data for RNN & build model

# ورودی را به شکل (batch, time_steps) داریم
# برای Embedding نیازی به one-hot نیست، فقط اندیس‌ها کافی است

model = models.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=8, input_length=seq_len),
    layers.SimpleRNN(32, return_sequences=False),
    layers.Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# آموزش
history = model.fit(
    X, y,
    epochs=30,
    batch_size=16,
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7391 - loss: 1.0449
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.5746
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0985 
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0241
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0134
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0096
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0078
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0062
Epoch 9/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0053
Epoch 10/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0044
Epoch 11/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0039
Epoch 12/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - l

In [3]:
# Cell 3: Sequence generation function

def generate_sequence(model, seed_text, length=20):
    """
    seed_text: رشته اولیه مثل "AB" یا "ABC"
    length: چند کاراکتر جدید تولید شود
    """
    # تبدیل seed به اندیس
    seed_ids = [char2idx[ch] for ch in seed_text]

    # اگر طول seed کمتر از seq_len است، از اول متن padding ساده می‌کنیم
    while len(seed_ids) < seq_len:
        seed_ids = [seed_ids[0]] + seed_ids  # خیلی ساده، اولین کاراکتر را تکرار می‌کنیم

    seed_ids = seed_ids[-seq_len:]  # فقط آخرین seq_len تا را نگه می‌داریم

    generated = list(seed_text)

    for _ in range(length):
        x_in = np.array(seed_ids)[np.newaxis, :]  # شکل (1, seq_len)
        probs = model.predict(x_in, verbose=0)[0]
        next_id = np.argmax(probs)  # می‌توانی نمونه‌برداری تصادفی هم بکنی
        next_char = idx2char[next_id]

        generated.append(next_char)

        # آپدیت پنجره
        seed_ids.append(next_id)
        seed_ids = seed_ids[-seq_len:]

    return "".join(generated)

# تست تولید دنباله
seed = "ABC"
generated_text = generate_sequence(model, seed_text=seed, length=30)
print("seed:", seed)
print("generated:", generated_text)


seed: ABC
generated: ABCCBCABCABCABCABCABCABCABCABCABC
